# webserv

## Directives

### http

In [ ]:
Syntax:	    http { ... }
Default:	—
Context:	main

An HTTP block specifies global configuration settings for the HTTP server. This can include settings such as the number of worker processes to run, the maximum number of connections allowed, and the location of log files.

Here is an example of an HTTP block in an NGINX configuration file:

In [ ]:
http {
    worker_processes 4;
    worker_rlimit_nofile 8192;
    client_max_body_size 128m;

    access_log /var/log/nginx/access.log;
    error_log /var/log/nginx/error.log;

    server {
        ...
    }
}

### server



In [ ]:
Syntax:	    server { ... }
Default:	—
Context:	http

The server directive is employed to define specific settings associated with an individual server. <br>
This configuration block allows the specification of parameters such as the listening port, server name, rules for handling requests, and other context-relevant settings for a virtual server. <br>
The server directive is crucial for setting up virtual servers within a web server instance, enabling the separate management of multiple websites or applications on the same physical server.

### listen

In [ ]:
Syntax:	    listen address[:port] [default_server] [ssl] [http2 | quic] [proxy_protocol] [setfib=number] [fastopen=number] [backlog=number] [rcvbuf=size] [sndbuf=size] [accept_filter=filter] [deferred] [bind] [ipv6only=on|off] [reuseport] [so_keepalive=on|off|[keepidle]:[keepintvl]:[keepcnt]];
            listen port [default_server] [ssl] [http2 | quic] [proxy_protocol] [setfib=number] [fastopen=number] [backlog=number] [rcvbuf=size] [sndbuf=size] [accept_filter=filter] [deferred] [bind] [ipv6only=on|off] [reuseport] [so_keepalive=on|off|[keepidle]:[keepintvl]:[keepcnt]];
            listen unix:path [default_server] [ssl] [http2 | quic] [proxy_protocol] [backlog=number] [rcvbuf=size] [sndbuf=size] [accept_filter=filter] [deferred] [bind] [so_keepalive=on|off|[keepidle]:[keepintvl]:[keepcnt]];
Default:	listen *:80 | *:8000;
Context:	server

The listen directive typically defines which IP address and port that the server block will respond to. By default, any server block that does not include a listen directive is given the listen parameters of 0.0.0.0:80 (or 0.0.0.0:8080 if Nginx is being run by a normal, non-root user). This allows these blocks to respond to requests on any interface on port 80, but this default value does not hold much weight within the server selection process.

The listen directive can be set to:

An IP address/port combo.
A lone IP address which will then listen on the default port 80.
A lone port which will listen to every interface on that port.
The path to a Unix socket.

For example, listen 80; indicates that the server will listen on all network interfaces on the default IP address for incoming requests on port 80. The listen directive is essential for configuring the listening behavior of the Nginx web server.

In [ ]:
server {
    listen 80; # corresponds to "listen *:80" meaning listen on all network interfaces on port 80.
    ...
}

Multiple listen directives can also be defined:

In [ ]:
server {
    listen localhost:80;
    listen 192.168.1.10:80;
    listen 192.168.1.10:8090;
    ...
}

In the Nginx configuration file, the default_server option specifies the default server to which a client request with an unknown domain and an empty host field will be forwarded. For instance, when a client writes out the server IP address into a browser or has many domains, such as linuxhint.com, test1.linuxhint.com, and test2.linuxhint.com, not all of them are mentioned in the Nginx configuration file.

If you have not added the “default_server” parameter to any virtual server, the first server will be considered the default.

In [ ]:
server {
    listen 80 default_server;
    ...
}

N.B. the default_server is only for one ip:port, consequently if there are virtual hosts with different ip:ports, there will be a default_server for each of them, whether specified or not, in this case the first one with this ip:port is designated

In [ ]:
server {
    listen localhost:80; # this is default_server for 127.0.0.1:80
    ...
}

server {
    listen localhost:80;
    ...
}

server {
    listen 192.168.24.24:8080;
    ...
}

server {
    listen 192.168.24.24:8080 default_server; # this is default_server for 192.168.24.24:8080
    ...
}

server {
    listen 192.168.42.42:80; # this is default_server for 192.168.42.42:80
    ...
}

### server_name


In [ ]:
Syntax:	    server_name name ...;
Default:	server_name "";
Context:	server

The server_name directive helps to route the HTTP request to the correct virtual host in the following way:
- the web server checks ip:host and checks all the virtual hosts matching this range;
- now checks the Host field of the HTTP request and compares it with the server_names of the different virtual hosts;
- if no server_name matches, the request is routed to the default_server virtual host.

The server_name directive accepts both single and multiple urls and ip.

Let's try two scenarios with two different HTTP requests, first request:
* GET / HTTP/1.1
* Host: 192.168.42.42
* User-Agent: curl/7.81.0

second request:
* GET / HTTP/1.1
* Host: another-site.org
* User-Agent: curl/7.81.0

In [ ]:
server {
    listen 80;
    server_name example.com www.example.com;
    ...
}

server {
    listen 80;
    server_name 127.0.0.1;
    ...
}

server {
    listen 80 default_server;
    server_name 192.168.42.42;
    ...
}

In these two scenarios, both requests will be routed to the third virtual host.

In [ ]:
Syntax:	    root path;
Default:	root html;
Context:	http, server, location, if in location

It is important to understand that Nginx will only evaluate the server_name directive when it needs to distinguish between server blocks that match to the same level of specificity in the listen directive. For instance, if example.com is hosted on port 80 of 192.168.1.10, a request for example.com will always be served by the first block in this example, despite the server_name directive in the second block.


In [ ]:
server {
    listen 192.168.1.10;
    ...
}

server {
    listen 80;
    server_name example.com;
    ...
}

### error_page

In [ ]:
Syntax:	    error_page code ... [=[response]] uri;
Default:	—
Context:	http, server, location

The error_page directive, allows for the configuration of error page handling for specific HTTP status codes. <br>
This directive defines which page to display when a particular HTTP error occurs. <br>
Here's an example of using the error_page directive:

In [ ]:
server {
    ...
    error_page 404 /custom_404.html;               # specifies that when a 404 error occurs, webserv should display the custom page "custom_404.html".
    error_page 500 502 503 504 /custom_error.html; # indicates that for errors 500, 502, 503, and 504, webserv should display the custom page "custom_error.html".

    location = /custom_error.html {                # directive configures the physical location of the custom page on the server, specifying the root directory.
        root /path/to/error/pages;
    }
    ...
}

### client_max_body_size

In [ ]:
Syntax:	    client_max_body_size size;
Default:	client_max_body_size 1m;
Context:	http, server, location

This directive is used to limit the maximum body size (payload) of an HTTP request that the server accepts from a client. <br>
This is particularly useful to prevent denial-of-service (DoS) attacks or to protect the server from requests with excessively large payloads that could exhaust the server's resources.

In [ ]:
server {
    listen 80;
    server_name example.com;

    client_max_body_size 10M;

    location /upload {
        # Configurations for handling upload requests
        ...
    }
}

### alias

In [ ]:
Syntax:	    alias path;
Default:	—
Context:	location

The alias directive is used to specify a mapping between a virtual path in the requested URL and a physical path on the server to the corresponding file or directory. <br>
This directive allows files to be served from a different location than the basic directory structure on the server.

Here is an example of how to use the alias directive:

In [ ]:
location /kapouet/ {                 # configures the search in the /tmp/www/ directory when the request is to /kapouet/.
    alias /tmp/www/;
    ...
}

location /kapouet/pouic/toto/pouet { # configures the search in the /tmp/www/pouic/to/pouet directory when the request is to /kapouet/pouic/to/pouet.
    alias /tmp/www/pouic/toto/pouet;
    ...
}


### location


In [ ]:
Syntax:	    location [ = | ~ | ~* | ^~ ] uri { ... }
            location @name { ... }
Default:	—
Context:	server, location

### root

In [ ]:
Syntax:	    root path;
Default:	root html;
Context:	http, server, location

### index


In [ ]:
Syntax:	    index file ...;
Default:    index index.html;
Context:	http, server, location

The index directive is used to specify the list of file names to search in a directory when a path matching a directory is requested. <br> The web server will search for these files in the order specified and respond with the first file found.


In [ ]:
location /my-directory/ {
    index index.html index.htm;  # Sets the search order of index files.
    ...
}


### autoindex


In [ ]:
Syntax:	    autoindex on | off;
Default:    autoindex off;
Context:	http, server, location

The autoindex directive is used to enable or disable the automatic display of the list of files in a directory when the request is directed to that directory and does not include a specific filename. <br>
In other words, if autoindex is enabled, the web server will generate a list of files and subdirectories present in the requested directory.

### limit_except


In [ ]:
Syntax:	    limit_except method ... { ... }
Default:	—
Context:	location

This directive allows you to prevent the use of all HTTP methods, except the ones that you explicitly allow. <br>
 Within a location block, you may want to restrict the use of some HTTP methods, such as forbidding clients from sending POST requests. <br>
 You need to define two elements: first, the methods that are not forbidden (the allowed methods; all others will be forbidden); and second, the audience that is affected by the restriction:

In [ ]:
location /admin/ { 
    limit_except GET POST { # Allow only GET and POST HTTP methods.
      allow 192.168.1.0/24; # Exclude from limitations 192.168.1.0/24.     
      deny all;             # Include everyone else in thats limitations.
    }
}

### rewrite

In [ ]:
Syntax:     rewrite regex replacement [flag];
Default:	—
Context:	server, location, if

The rewrite directive allows URLs or HTTP requests to be dynamically modified. When a match is found between the requested URL and the string specified in the rewrite directive, the request is forwarded to the new URL with an HTTP 301 (Moved Permanently) response code.

The HTTP 301 response code informs the browser and other user agents that the requested resource has been permanently moved to a new address. <br> 
As a result, the browser should automatically refresh its cache and store the new URL. <br> 
This permanent redirection is useful when you want to indicate to search engines and users that the original URL is no longer valid and should be replaced by the new URL indicated in the rewrite.

In [ ]:
location /old {
    rewrite /old /new;
}

### include


In [ ]:
Syntax:	    include file | mask;
Default:	—
Context:	any

The include directive is used to include the contents of other configuration files within the main configuration file. <br>
This allows the configuration to be more modularly organized and makes it easier to manage configuration files.

In [ ]:
http {
    include /etc/nginx/mime.types;
    include /etc/nginx/conf.d/*.conf;

    server {
        listen 80;
        server_name example.com;
        include /etc/nginx/conf.d/server_config/*.conf;  # Includes other server-specific configuration files.
        ...
    }
    ...
}